In [70]:
import pyterrier as pt
if not pt.started():
  pt.init()

# User-based collaborative filtering
import pandas as pd
import numpy as np

C:\Users\Fortu\AppData\Local\Temp\ipykernel_28528\1874076957.py:2: DeprecationWarning: Call to deprecated function (or staticmethod) started. (use pt.java.started() instead) -- Deprecated since version 0.11.0.
  if not pt.started():


In [71]:
df = pd.read_csv('./songs_sample.csv')

In [72]:
df.shape

(7357, 7)

In [73]:
df.head()

,listen_time,userid,title,release,artist_name,year,songid
0,1,27018,Harder_Better_Faster_Stronger,Discovery,Daft Punk,2007,97
1,3,27018,Phantom_Part_1.5_(Album_Version),A Cross The Universe,Justice,0,122
2,1,27018,I_Got_Mine,Attack & Release,The Black Keys,2008,139
3,1,27018,Face_To_Face_(Cosmo_VItelli_Remix),Daft Club,Daft Punk,2003,149
4,2,27018,Pogo,Idealism,Digitalism,2007,189


In [74]:
# data normalization
from sklearn.preprocessing import MinMaxScaler

In [ ]:
col = 'listen_time'
df[col] = MinMaxScaler().fit_transform(df[col].values.reshape(-1, 1))

In [76]:
# Create user matrix: contains relationships among users and items
user_matrix = pd.pivot_table(df, values='listen_time', index='userid', columns='songid')

In [77]:
user_matrix.head()

songid,0,1,4,8,9,10,11,12,15,16,...,9984,9985,9988,9989,9990,9993,9994,9996,9997,9998
userid,,,,,,,,,,,,,,,,,,,,,
457,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
902,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1041,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1159,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3790,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [87]:
user_matrix = user_matrix.fillna(0)

In [ ]:
user_matrix.head()

songid,0,1,4,8,9,10,11,12,15,16,...,9984,9985,9988,9989,9990,9993,9994,9996,9997,9998
userid,,,,,,,,,,,,,,,,,,,,,
457,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
902,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1041,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1159,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3790,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [80]:
user_matrix.shape

(74, 7357)

In [81]:
from sklearn.metrics.pairwise import cosine_similarity
import operator

In [82]:
def similar_users(user_id, matrix, k=1):
  """
  Find the most similar users to a given user based on cosine similarity.
  Parameters:
  user_id (int or str): The ID of the user for whom to find similar users.
  matrix (pd.DataFrame): A pandas DataFrame where rows represent users and columns represent features.
  k (int, optional): The number of similar users to return. Default is 1.
  Returns:
  list: A list of user IDs that are most similar to the given user.
  """
  # Create a df of just the current user
  user = matrix[matrix.index == user_id]
  # And one of all other users
  other_users = matrix[matrix.index != user_id]
  # calculate cosine similarity between user and each other user one by one
  similarities = cosine_similarity(user, other_users)[0].tolist()
  # Create list of indices of these users to get the data quickly
  indices = other_users.index.tolist()
  # Create key/values pairs of user index and their similarity
  index_similarity = dict(zip(indices, similarities))
  # Sort by similarity
  index_similarity_sorted = sorted(index_similarity.items(), key=operator.itemgetter(1))
  index_similarity_sorted.reverse()
  # Get the top k highest similarities
  top_users_similarities = index_similarity_sorted[:k]
  simusers = [u[0] for u in top_users_similarities]

  return simusers

In [83]:
# This is what we want to do
bc = 457
sm = similar_users(bc, user_matrix, k=5)
print(sm)

[75144, 74130, 71183, 71020, 70967]


In [100]:
# Create a function to recommend songs to a user
def recommend_items(userid,similar_user_indices,matrix,items):
  # load vector for the similar users
  similar_users = matrix[matrix.index.isin(similar_user_indices)]
  # calculate the mean listen time for each song
  similar_users = similar_users.mean(axis=0)
  # convert to dataframe so its easy to sort and filter
  similar_users_df = pd.DataFrame(similar_users, columns=['mean'])
  # load vector for the current user
  user = matrix[matrix.index == userid]
  # Transpose it so its easier to filter
  songs_listened = user.transpose()
  songs_listened.columns = ['listen_time']
  # select songs that the user has not listened to
  songs_listened = songs_listened[songs_listened['listen_time']==0]
  # generate a list of songs the user has not listened to
  songs_nl = songs_listened.index.tolist()
  # score those songs by filtering the similar users vector to just the songs the current user has not listened to
  scores = similar_users_df[similar_users_df.index.isin(songs_nl)]
  # sort the scores and return the top items
  recommendations = scores.sort_values('mean', ascending=False)
  topsongs = recommendations.head(items)
  top_n_songs_indices = recommendations.index.tolist()
  song_information = df[df['songid'].isin(top_n_songs_indices)]

  return topsongs, song_information

In [102]:
# This is what we want to do
x, y = recommend_items(457,sm,user_matrix,10)

x

,mean
songid,
2408,0.048684
1498,0.042105
6369,0.034211
5781,0.023684
3934,0.021053
7515,0.021053
3404,0.017105
8073,0.015789
6363,0.015789


In [103]:
y.head()

,listen_time,userid,title,release,artist_name,year,songid
0,0.000000,27018,Harder_Better_Faster_Stronger,Discovery,Daft Punk,2007,97
1,0.013158,27018,Phantom_Part_1.5_(Album_Version),A Cross The Universe,Justice,0,122
2,0.000000,27018,I_Got_Mine,Attack & Release,The Black Keys,2008,139
3,0.000000,27018,Face_To_Face_(Cosmo_VItelli_Remix),Daft Club,Daft Punk,2003,149
4,0.006579,27018,Pogo,Idealism,Digitalism,2007,189
